## Error Handling and Recovery

### Understanding `Result` and `Option`

In [3]:
let result: Result<i32, &str> = Ok(42);
let option: Option<i32> = Some(42);
option

Some(42)

#### Handling Errors with `Result`

In [5]:
fn divide(a: i32, b: i32) -> Result<i32, String> {
    if b == 0 {
        return Err(format!("Cannot divide {} by zero.", a));
    }

    Ok(a / b)
}

let result = divide(10, 2);

match result {
    Ok(value) => println!("Result: {}", value),
    Err(error) => eprintln!("Error: {}", error),
}

Result: 5


()

#### Handling Errors with `Option`

In [7]:
fn find_element(arr: &[i32], target: i32) -> Option<usize> {
    for (i, &element) in arr.iter().enumerate() {
        if element == target {
            return Some(i);
        }
    }

    None
}

let numbers = [1, 2, 3, 4, 5];
let target = 3;

match find_element(&numbers, target) {
    Some(index) => println!("Found at index: {}", index),
    None => println!("Element not found."),
}

Found at index: 2


()

### Techniques for Error Propagation and Handling Multiple Errors

#### The `?` Operator

In [8]:
use std::fs::File;
use std::io::Read;

fn read_file_contents(file_path: &str) -> Result<String, std::io::Error> {
    let mut file = File::open(file_path)?;
    let mut contents = String::new();
    file.read_to_string(&mut contents)?;
    Ok(contents)
}

fn main() -> Result<(), Box<dyn std::error::Error>> {
    let file_path = "example.txt";
    let contents = read_file_contents(file_path)?;
    println!("File contents: {}", contents);
    Ok(())
}

main()

Err(Os { code: 2, kind: NotFound, message: "No such file or directory" })

#### Handling Multiple Errors with `Result`

In [9]:
fn parse_and_square(input: &str) -> Result<i32, Box<dyn std::error::Error>> {
    let number = input.parse::<i32>()?;
    let squared = number * number;
    Ok(squared)
}

fn main() -> Result<(), Box<dyn std::error::Error>> {
    let input = "9";
    let result = parse_and_square(input)?;

    println!("Result: {}", result);
    Ok(())
}

main()

Result: 81


Ok(())

#### Error Propagation with `Result` and `Option`

In [11]:
fn find_element(arr: &[i32], target: i32) -> Option<usize> {
    arr.iter().position(|&x| x == target)
}

fn divide(dividend: i32, divisor: i32) -> Result<i32, String> {
    if divisor == 0 {
        Err("Division by zero".to_string())
    } else {
        Ok(dividend / divisor)
    }
}

fn find_and_divide(arr: &[i32], target: i32, divisor: i32) -> Result<f64, String> {
    let index = find_element(arr, target)
        .ok_or_else(|| "Element not found.".to_string())?;

    let result = divide(arr[index], divisor)?;

    Ok(result as f64)
}

fn main() {
    let numbers = [1, 2, 3, 4, 5];
    let target = 3;
    let divisor = 2;

    match find_and_divide(&numbers, target, divisor) {
        Ok(result) => println!("Result: {}", result),
        Err(error) => eprintln!("Error: {}", error),
    }
}

main()

Result: 1


()

### Creating Custom Error Types

In [13]:
use std::fs::File;
use std::io::Read;
use std::io::Error as IoError;

#[derive(Debug)]
enum FileReadError {
    FileNotFound,
    IoError(IoError),
}

impl From<IoError> for FileReadError {
    fn from(error: IoError) -> Self {
        if error.kind() == std::io::ErrorKind::NotFound {
            FileReadError::FileNotFound
        } else {
            FileReadError::IoError(error)
        }
    }
}

fn read_file_contents(file_path: &str) -> Result<String, FileReadError> {
    let mut file = File::open(file_path).map_err(FileReadError::from)?;

    let mut contents = String::new();
    file.read_to_string(&mut contents)?;
    Ok(contents)
}

fn main() -> Result<(), Box<dyn std::error::Error>> {
    let file_path = "nonexistent.txt";
    match read_file_contents(file_path) {
        Ok(contents) => println!("File contents: {}", contents),
        Err(FileReadError::FileNotFound) => eprintln!("File not found."),
        Err(FileReadError::IoError(e)) => eprintln!("IO Error: {}", e),
    }
    Ok(())
}

main()

File not found.


Ok(())

### Advanced Error Handling

#### The `anyhow` Library

In [14]:
:dep anyhow = {version = "1.0.75"}

In [20]:
use anyhow::{anyhow, Result};

fn divide(a: i32, b: i32) -> Result<i32> {
    if b == 0 {
        return Err(anyhow!("Cannot divide by zero."));
    }

    Ok(a / b)
}

fn main() -> Result<()> {
    let result = divide(10, 0).unwrap();
    println!("Result: {}", result);
    Ok(())
}

main()
// TODO: Report a bug

Error: `?` couldn't convert the error to `String`

#### Custom Error Types with `thiserror`

In [24]:
:dep thiserror = {version = "1.0.50"}

In [25]:
use thiserror::Error;

#[derive(Error, Debug)]
enum MyError {
    #[error("File not found: {0}")]
    FileNotFound(String),
    #[error("IO error: {0}")]
    IoError(#[from] std::io::Error),
}

fn read_file_contents(file_path: &str) -> Result<String, MyError> {
    let result = std::fs::read_to_string(file_path);

    match result {
        Ok(contents) => Ok(contents),
        Err(e) => {
            if e.kind() == std::io::ErrorKind::NotFound {
                Err(MyError::FileNotFound(file_path.to_string()))
            } else {
                Err(MyError::IoError(e))
            }
        }
    }
}

fn main() -> Result<(), MyError> {
    let file_path = "example.txt";
    let contents = read_file_contents(file_path)?;
    println!("File contents: {}", contents);
    Ok(())
}

main()

Err(FileNotFound("example.txt"))

### Asynchronous Error Handling

#### Async Functions and Results

In [28]:
:dep reqwest = {version = "0.11.22"}

In [27]:
:dep tokio = { version = "1.34.0", features = ["full"] }

In [33]:
use reqwest;

async fn fetch_data() -> Result<String, reqwest::Error> {
    let response = reqwest::get("https://www.google.com/").await?;
    let body = response.text().await?;
    Ok(body)
}

#[tokio::main]
async fn main() -> Result<(), Box<dyn std::error::Error>> {
    // You can run asynchronous code here if needed.
    let result = fetch_data().await;

    match result {
        Ok(body) => {
            println!("Fetched data:\n{}", body);
        }
        Err(err) => {
            eprintln!("Error fetching data: {:?}", err);
        }
    }

    Ok(())
}

main()

Fetched data:
<!doctype html><html dir="rtl" itemscope="" itemtype="http://schema.org/WebPage" lang="ar-LB"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/logos/doodles/2023/lebanon-independence-day-2023-6753651837109971-law.gif" itemprop="image"><meta content="&#1593;&#1610;&#1583; &#1575;&#1587;&#1578;&#1602;&#1604;&#1575;&#1604; &#1604;&#1576;&#1606;&#1575;&#1606; 2023" property="twitter:title"><meta content="&#1593;&#1610;&#1583; &#1575;&#1587;&#1578;&#1602;&#1604;&#1575;&#1604; &#1604;&#1576;&#1606;&#1575;&#1606; #GoogleDoodle" property="twitter:description"><meta content="&#1593;&#1610;&#1583; &#1575;&#1587;&#1578;&#1602;&#1604;&#1575;&#1604; &#1604;&#1576;&#1606;&#1575;&#1606; #GoogleDoodle" property="og:description"><meta content="summary_large_image" property="twitter:card"><meta content="@GoogleDoodles" property="twitter:site"><meta content="https://www.google.com/logos/doodles/2023/lebanon-independence-day-2023-6753651837109971-2xa.g

Ok(())

#### Custom Error Types for Async Code

In [ ]:
use std::io;

#[derive(Debug)]
enum MyError {
    Network(reqwest::Error),
    Io(io::Error),
}

async fn fetch_data() -> Result<String, MyError> {
    // ...
}

#### Handling Concurrency Errors

In [38]:
use tokio::sync::mpsc;
use std::error::Error;

#[tokio::main]
async fn main() -> Result<(), Box<dyn Error>> {
    let (tx, mut rx) = mpsc::channel(32);

    for i in 0..4 {
        let tx = tx.clone();
        tokio::spawn(async move {
            let result = do_work(i).await;
            tx.send(result).await.expect("Send failed");
        });
    }

    for _ in 0..4 {
        if let Some(result) = rx.recv().await {
            println!("Received result: {:?}", result);
        }
    }

    Ok(())
}

async fn do_work(task_id: i32) -> i32 {
    tokio::time::sleep(std::time::Duration::from_secs(1)).await;

    task_id * 2
}

main()
// TODO: Report a bug

Error: cannot find derive macro `Error` in this scope

Error: cannot find attribute `error` in this scope

Error: cannot find attribute `error` in this scope

Error: cannot find attribute `from` in this scope

#### Error Handling in Web Applications

In [39]:
:dep warp = { version = "0.3.6" }

In [40]:
use warp::Filter;

#[tokio::main]
async fn main() {
    let hello = warp::path!("hello" / "world")
        .map(|| {
            // Your web application logic here
            warp::reply::html("Hello, world!")
        });

    let routes = hello.with(warp::log("myapp::api"));
    warp::serve(routes).run(([127, 0, 0, 1], 3030)).await;
}

main()

Error: Subprocess terminated with status: signal: 9 (SIGKILL)

### Putting It All Together

#### File Parsing and Error Handling

In [43]:
:dep csv = { version = "1.3.0" }

In [44]:
use std::error::Error;
use std::fs::File;
use std::io::{self, BufRead, BufReader};

#[derive(Debug)]
enum CsvError {
    Io(io::Error),
    ParseError(csv::Error),
}

impl From<io::Error> for CsvError { // <1>
    fn from(error: io::Error) -> Self {
        CsvError::Io(error)
    }
}

impl From<csv::Error> for CsvError { // <1>
    fn from(error: csv::Error) -> Self {
        CsvError::ParseError(error)
    }
}

fn parse_csv(file_path: &str) -> Result<(), CsvError> { // <2>
    let file = File::open(file_path)?;
    let reader = BufReader::new(file);

    for line in reader.lines() {
        let line = line?;
        let record = csv::Reader::from_reader(line.as_bytes()).deserialize::<(String, i32)>();
        // Process the record here
    }

    Ok(())
}

parse_csv("esports_data.csv")

Error: cannot find derive macro `Error` in this scope

Error: cannot find attribute `error` in this scope

Error: cannot find attribute `error` in this scope

Error: cannot find attribute `from` in this scope

### Error Handling in Command-Line Applications

In [48]:
:dep clap = { version = "4.4.8", features = ["derive"] }

In [49]:
use clap::{Parser, Subcommand};
use clap::Args;

#[derive(Parser)]
#[clap(author, version, about = "My Rust CLI App", long_about = None)]
struct Cli {
    #[command(subcommand)]
    command: Option<Commands>,
}

#[derive(Subcommand)]
enum Commands {
    Input(Input),
    Output(Output),
}

#[derive(Args)]
struct Input {
    file_name: Option<String>,
}

#[derive(Args)]
struct Output {
    file_name: Option<String>,
}

fn main() -> Result<(), Box<dyn std::error::Error>>  {
    let args = Cli::parse();
    match args.command {
        Some(Commands::Input(file)) => {
            match file.file_name {
                Some(ref file_name) => {
                    // Perform the application logic here
                }
                None => {
                    println!("Please provide an input file name");
                }
            }
        }
        Some(Commands::Output(file)) => {
            match file.file_name {
                Some(ref file_name) => {
                    // Perform the application logic here
                }
                None => {
                    println!("Please provide an output file name");
                }
            }
        }
        None => todo!()

    }

    // Perform the application logic here
    Ok(())
}

main()

// TODO: Report a bug/edge case.

thread '<unnamed>' panicked at src/lib.rs:67:17:
not yet implemented
stack backtrace:
   0: rust_begin_unwind
             at /rustc/cd674d61790607dfb6faa9d754bd3adfa13aea7c/library/std/src/panicking.rs:597:5
   1: core::panicking::panic_fmt
             at /rustc/cd674d61790607dfb6faa9d754bd3adfa13aea7c/library/core/src/panicking.rs:72:14
   2: core::panicking::panic
             at /rustc/cd674d61790607dfb6faa9d754bd3adfa13aea7c/library/core/src/panicking.rs:127:5
   3: ctx::main
   4: std::panicking::try
   5: run_user_code_22
   6: evcxr::runtime::Runtime::run_loop
   7: evcxr::runtime::runtime_hook
   8: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


### Handling Signals and Interruptions

In [50]:
:dep ctrlc = { version = "3.4.1" }

In [52]:
use ctrlc;
use std::sync::atomic::{AtomicBool, Ordering};
use std::sync::Arc;

fn main() -> Result<(), Box<dyn std::error::Error>> {
    let running = Arc::new(AtomicBool::new(true));
    let r = running.clone();

    ctrlc::set_handler(move || {
        r.store(false, Ordering::SeqCst);
        println!("Just got interrupted!");
    })?;

    // Perform application logic here

    while running.load(Ordering::SeqCst) {
        // Keep the application running
    }

    Ok(())
}

main()

Error: Subprocess terminated with status: signal: 9 (SIGKILL)

### Error Handling in File I/O

#### Reading and Writing Files

In [53]:
use std::fs::File;
use std::io::{Read, Write};

fn main() -> Result<(), Box<dyn std::error::Error>> {
    let file_path = "example.txt";
    let mut file = File::open(file_path)?;

    let mut contents = String::new();
    file.read_to_string(&mut contents)?;

    // Process and manipulate the file contents here

    let mut output_file = File::create("output.txt")?;
    output_file.write_all(contents.as_bytes())?;

    Ok(())
}

main()

Err(Os { code: 2, kind: NotFound, message: "No such file or directory" })

#### Working with Directories

In [54]:
use std::fs;
use std::path::Path;

fn main() -> Result<(), Box<dyn std::error::Error>> {
    let dir_path = "my_directory";

    if !Path::new(dir_path).exists() {
        fs::create_dir(dir_path)?;
    }

    // Perform operations within the directory here

    Ok(())
}

main()

Ok(())

### Error Handling in Network Programming

#### Making HTTP Requests

In [57]:
use reqwest;

#[tokio::main]
async fn main() -> Result<(), reqwest::Error> {
    let response = reqwest::get("https://google.com").await?;

    if response.status().is_success() {
        // Process the response body
        let body = response.text().await?;
        println!("Response body:\n{}", body);
    } else {
        eprintln!("HTTP request failed: {:?}", response.status());
    }

    Ok(())
}

main()

Response body:
<!doctype html><html dir="rtl" itemscope="" itemtype="http://schema.org/WebPage" lang="ar-LB"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/logos/doodles/2023/lebanon-independence-day-2023-6753651837109971-law.gif" itemprop="image"><meta content="&#1593;&#1610;&#1583; &#1575;&#1587;&#1578;&#1602;&#1604;&#1575;&#1604; &#1604;&#1576;&#1606;&#1575;&#1606; 2023" property="twitter:title"><meta content="&#1593;&#1610;&#1583; &#1575;&#1587;&#1578;&#1602;&#1604;&#1575;&#1604; &#1604;&#1576;&#1606;&#1575;&#1606; #GoogleDoodle" property="twitter:description"><meta content="&#1593;&#1610;&#1583; &#1575;&#1587;&#1578;&#1602;&#1604;&#1575;&#1604; &#1604;&#1576;&#1606;&#1575;&#1606; #GoogleDoodle" property="og:description"><meta content="summary_large_image" property="twitter:card"><meta content="@GoogleDoodles" property="twitter:site"><meta content="https://www.google.com/logos/doodles/2023/lebanon-independence-day-2023-6753651837109971-2xa.

Ok(())

#### Building Network Services

In [59]:
use std::net::{TcpListener, TcpStream};
use std::io::{Read, Write};
use std::thread;

fn handle_client(mut stream: TcpStream) -> Result<(), Box<dyn std::error::Error>> {
    // Handle client logic here
    println!("Client connected: {:?}", stream.peer_addr()?);

    let mut buffer = [0; 1024];
    loop {
        let bytes_read = stream.read(&mut buffer)?;

        if bytes_read == 0 {
            // Connection closed
            println!("Client disconnected: {:?}", stream.peer_addr()?);
            break;
        }

        let data = &buffer[..bytes_read];
        println!("Received from client {:?}: {:?}", stream.peer_addr()?, String::from_utf8_lossy(data));

        // Process data received from the client
        // Example: Echo the data back to the client
        stream.write_all(data)?;
    }

    Ok(())
}

fn main() -> Result<(), Box<dyn std::error::Error>> {
    let listener = TcpListener::bind("127.0.0.1:8080")?; // Change the address and port as needed
    println!("Server listening on: {:?}", listener.local_addr()?);

    for stream in listener.incoming() {
        match stream {
            Ok(stream) => {
                println!("Accepted connection from: {:?}", stream.peer_addr()?);

                thread::spawn(move || {
                    if let Err(err) = handle_client(stream) {
                        eprintln!("Error handling client: {:?}", err);
                    }
                });
            }
            Err(e) => {
                eprintln!("Failed to accept client connection: {:?}", e);
            }
        }
    }

    Ok(())
}

main()

// In a new terminal, run:
// $ telnet 127.0.0.1 8080

// Trying 127.0.0.1...
// Connected to 127.0.0.1.
// Escape character is '^]'.
// Hello
// Hello

Server listening on: 127.0.0.1:8080
Accepted connection from: 127.0.0.1:43266
Client connected: 127.0.0.1:43266
Received from client 127.0.0.1:43266: "Hello\r\n"
Received from client 127.0.0.1:43266: "\r\n"
Received from client 127.0.0.1:43266: "����\u{6}"
Received from client 127.0.0.1:43266: "��\u{6}"
Received from client 127.0.0.1:43266: "����\u{6}"
Received from client 127.0.0.1:43266: "��\u{6}"
Received from client 127.0.0.1:43266: "����\u{6}"
Received from client 127.0.0.1:43266: "��\u{6}"
Received from client 127.0.0.1:43266: "����\u{6}"
Received from client 127.0.0.1:43266: "��\u{6}"
Received from client 127.0.0.1:43266: "����\u{6}"
Received from client 127.0.0.1:43266: "��\u{6}"
Received from client 127.0.0.1:43266: "����\u{6}"
Received from client 127.0.0.1:43266: "��\u{6}"


Error: Subprocess terminated with status: signal: 9 (SIGKILL)

#### Error Handling in Multithreaded and Concurrent Code

In [60]:
use std::sync::{Arc, Mutex};
use std::thread;

fn process_data(data: Arc<Mutex<Vec<u32>>>) -> Result<(), Box<dyn std::error::Error>> {
    // Lock the data for exclusive access within this thread
    let mut data_lock = data.lock().unwrap();

    // Process the data concurrently
    for i in 0..4 {
        data_lock.push(i);
    }

    // Data is automatically unlocked when `data_lock` goes out of scope

    Ok(())
}

fn main() -> Result<(), Box<dyn std::error::Error>> {
    // Create a shared data structure (a vector of u32) using Arc and Mutex
    let data = Arc::new(Mutex::new(Vec::new()));

    // Create a vector to store thread handles
    let mut handles = vec![];

    // Spawn multiple threads to process data concurrently
    for _ in 0..4 {
        let data_clone = Arc::clone(&data); // Clone the Arc

        let handle = thread::spawn(move || {
            if let Err(err) = process_data(data_clone) {
                eprintln!("Error processing data: {:?}", err);
            }
        });

        handles.push(handle);
    }

    // Wait for all threads to complete
    for handle in handles {
        handle.join().unwrap();
    }

    // Access the shared data after processing
    let shared_data = {
        let data_lock = data.lock().unwrap();
        data_lock.clone()
    };

    // Print the content of the vector
    for item in &*shared_data {
        println!("Processed data item: {}", item);
    }

    Ok(())
}

main()

Processed data item: 0
Processed data item: 1
Processed data item: 2
Processed data item: 3
Processed data item: 0
Processed data item: 1
Processed data item: 2
Processed data item: 3
Processed data item: 0
Processed data item: 1
Processed data item: 2
Processed data item: 3
Processed data item: 0
Processed data item: 1
Processed data item: 2
Processed data item: 3


Ok(())

#### Cross-Thread Communication

In [62]:
use std::thread;
use std::sync::mpsc;

fn main() -> Result<(), Box<dyn std::error::Error>> {
    let (tx, rx) = mpsc::channel();

    let worker = thread::spawn(move || {
        // Worker thread logic
        tx.send("Task completed").unwrap();
    });

    // Main thread logic

    let result = rx.recv()?;
    println!("Received result: {}", result);

    worker.join().unwrap();

    Ok(())
}

main()

Received result: Task completed


Ok(())

### Testing and Error Handling

#### Writing Error Tests

In [64]:
fn divide(dividend: i32, divisor: i32) -> Result<i32, String> {
    if divisor == 0 {
        Err(format!("Cannot divide {} by zero.", dividend))
    } else {
        Ok(dividend / divisor)
    }
}

#[test]
fn test_divide_by_zero() {
    let result = divide(10, 0);

    assert!(result.is_err());
    assert_eq!(result.unwrap_err(), "Cannot divide 10 by zero.");
}

test_divide_by_zero()

// TODO: file feature request?

Error: cannot find function `test_divide_by_zero` in this scope

#### Property-Based Testing

In [65]:
:dep proptest = { version = "1.0.0" }

In [67]:
use proptest::prelude::*;

fn divide(dividend: i32, divisor: i32) -> Result<i32, String> {
    if divisor == 0 {
        Err(format!("Cannot divide {} by zero.", dividend))
    } else {
        Ok(dividend / divisor)
    }
}

proptest! {
    #[test]
    fn test_divide_property_based(input in 1..100, divisor in 1..100) {
        if divisor == 0 {
            let result = divide(input, divisor);
            assert!(result.is_err());
            assert_eq!(result.unwrap_err(), format!("Cannot divide {} by zero.", input));
        } else {
            let result = divide(input, divisor);
            assert!(result.is_ok());
            assert_eq!(result.unwrap(), input / divisor);
        }
    }
}


Error: expected expression, found keyword `fn`

---
---